In [14]:
from datasets import Dataset

# dataset_1 = Dataset.from_json("datasets/llm1_dataset.jsonl")
dataset_2 = Dataset.from_json("datasets/llm2_dataset.jsonl")
# dataset_3 = Dataset.from_json("datasets/llm3_dataset.jsonl")
# all_datasets = {"llm1": dataset_1, "llm2": dataset_2, "llm3": dataset_3}

In [2]:
print(dataset_2)

Dataset({
    features: ['messages'],
    num_rows: 25548
})


In [5]:
Dataset.from_json("datasets/llm2_sharegpt.jsonl")[0]

{'conversations': [{'from': 'system',
   'value': "You, being an honest and intelligent assistant, are tasked with creating steps for accomplishment. There's no requirement for executing the functions; instead, organize them logically using your cognitive abilities. The output format should be strictly in JSON."},
  {'from': 'human',
   'value': '{"query": "What are the latest developments in quantum computing as of 2023?", "steps": ["RESEARCH: Investigate the most recent advancements in quantum computing technology"], "functions": [{"name": "SCI_TECH_INNOVATIONS", "description": "Provides details on recent scientific and technological advancements in various fields.", "parameters": {"type": "object", "properties": [{"name": "field", "type": "string", "description": "The specific field of science or technology to get updates on, such as \'Biotechnology\', \'Artificial Intelligence\', or \'Renewable Energy\'."}, {"name": "date_range", "type": "string", "description": "The range of dates

In [6]:
from helpers.formatter import MessagesFormatter, DatasetFormatter

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [8]:
mf = MessagesFormatter(dataset_1[:1]['messages'], separator="\n").format().tokenize(tokenizer)

In [34]:
print(mf.formatted_messages[0])

<<SYS>> Embodying the role of an earnest and brainy assistant who can reframe questions into actionable components, you face a question framed in JSON. You are to issue a response as a JSON object, consisting of can_i_answer (true illustrates the question is answerable independently, or false if it leans on external resources) and tasks, enumerating procedures to tackle the question using external sources, contingent on can_i_answer being false. <<SYS>>
[INST] {"question": "2023 Nobel Prize winners in Physics announced?"} [/INST]
{"can_i_answer": false, "tasks": ["SEARCH: Check for the announcement of Nobel Prize winners in Physics for 2023"], "response": null}


In [9]:
mf1 = MessagesFormatter(
    [
        [
            {"role": "system", "content": "systemmessage"},
            {"role": "user", "content": "userinput"},
            {"role": "assistant", "content": "assistantoutput"},
        ]
    ],
    separator="\n",
    system_template="<|im_start|>system\n{system}<|im_end|>",
    user_template="<|im_start|>user\n{user}<|im_end|>",
    assistant_template="<|im_start|>assistant\n{assistant}<|im_end|>",
).format().tokenize(tokenizer)

In [14]:
import json
import time
from pathlib import Path
from itertools import chain


p = Path(r"D:\SmartLLM\data\functions")
funcs = list(chain(*[json.load(open(p)) for p in p.rglob("*.json")]))

In [16]:
def convert_to_alpaca_format(d: dict):
    messages = d.pop("messages", [])
    return {"system": messages[0]["content"], "input": messages[1]["content"], "output": messages[2]["content"]}

dataset_2.map(convert_to_alpaca_format).to_json("datasets/llm2_alpaca.jsonl")

Creating json from Arrow format:   0%|          | 0/26 [00:00<?, ?ba/s]

Creating json from Arrow format: 100%|██████████| 26/26 [00:04<00:00,  5.51ba/s]


136208104

In [89]:
prompt = '<<SYS>> Embodying the role of an earnest and brainy assistant who can reframe questions into actionable components, you face a question framed in JSON. You are to issue a response as a JSON object, consisting of can_i_answer (true illustrates the question is answerable independently, or false if it leans on external resources) and tasks, enumerating procedures to tackle the question using external sources, contingent on can_i_answer being false. <<SYS>>\n[INST] {"question": "2023 Nobel Prize winners in Physics announced?"} [/INST]\n'
output = '<s> <<SYS>> Embodying the role of an earnest and brainy assistant who can reframe questions into actionable components, you face a question framed in JSON. You are to issue a response as a JSON object, consisting of can_i_answer (true illustrates the question is answerable independently, or false if it leans on external resources) and tasks, enumerating procedures to tackle the question using external sources, contingent on can_i_answer being false. <<SYS>>\n[INST] {"question": "2023 Nobel Prize winners in Physics announced?"} [/INST]\n{"can_i_answer": false, "tasks": ["SEARCH: Check for the announcement of Nobel Prize winners in Physics for 2023"], "response": null} </s>'
extract = '{"question": "2023 Nobel Prize winners in Physics announced?"} [/INST] {"can_i_answer": false, "tasks": ["SEARCH: Check for the announcement of Nobel Prize winners in Physics for 2023"], "response": null}'

In [112]:
mf.formatted_messages[0]

mf.base_format

attr_names = ["system", "user", "assistant"]

for attr_name in attr_names:
    print(f"{attr_name}: {getattr(mf, f'{attr_name}_template')}")

system: <<SYS>> {system} <<SYS>>
user: [INST] {human} [/INST]
assistant: {gpt}


In [3]:
from parse import parse


parse(mf.base_format, mf.base_format).named

NameError: name 'mf' is not defined

In [245]:
parse("{name} is {adjective}, I {} her!", "Hanna is lovely, I her!")

In [2]:
template = '{system} <<SYS>>\n[INST] {human} [/INST]\n{gpt}'
formatted_text = 'system <<SYS>>\n[INST] human [/INST]\ngpt'

template = "{name} is {adjective}, I {verb} her {items}!"
formatted_text = "Hanna is lovely, I love her eyes!"

In [13]:
from helpers.text_utils import TextUtils

TextUtils.parse_to_dict(mf.base_format, mf.formatted_messages[0])

{'system': 'Embodying the role of an earnest and brainy assistant who can reframe questions into actionable components, you face a question framed in JSON. You are to issue a response as a JSON object, consisting of can_i_answer (true illustrates the question is answerable independently, or false if it leans on external resources) and tasks, enumerating procedures to tackle the question using external sources, contingent on can_i_answer being false.',
 'user': '{"question": "2023 Nobel Prize winners in Physics announced?"}',
 'assistant': '{"can_i_answer": false, "tasks": ["SEARCH: Check for the announcement of Nobel Prize winners in Physics for 2023"], "response": null}'}

In [36]:
from itertools import chain
from datasets import Dataset
from models.inputs import StepsInput
from models.outputs import StepsOutput

dataset_2 = Dataset.from_json("datasets/llm2_dataset.jsonl")
d = {
    StepsInput.model_validate_json(
        x["messages"][1]["content"]
    ): StepsOutput.model_validate_json(x["messages"][2]["content"])
    for x in dataset_2
}

inconsistent = {
    f"{hash(i)}{hash(o)}": (
        i.query,
        set(chain(*[x.available_tools for x in o.steps]))
        - set([f.name for f in i.functions]),
    )
    for ix, (i, o) in enumerate(d.items())
    if set(chain(*[x.available_tools for x in o.steps]))
    - set([f.name for f in i.functions])
    != set()
}
indexes_to_remove = list(inconsistent)
df_2 = dataset_2.to_pandas()
# del dataset_2
# df_2_dict = df_2.drop(indexes_to_remove).to_dict(orient="records")
# dataset_2 = Dataset.from_list(
#     [{"messages": list(x.get("messages", []))} for x in df_2_dict]
# )

In [37]:
df_2['hash'] = df_2['messages'].apply(lambda x: f"{hash(StepsInput.model_validate_json(x[1]['content']))}{hash(StepsOutput.model_validate_json(x[2]['content']))}")

In [50]:
dataset_2 = Dataset.from_pandas(df_2.set_index('hash', drop=True).drop(indexes_to_remove).reset_index(drop=True))

In [54]:
dataset_2.to_json("datasets/llm2_dataset.jsonl")

Creating json from Arrow format:   0%|          | 0/26 [00:00<?, ?ba/s]

Creating json from Arrow format: 100%|██████████| 26/26 [00:02<00:00,  9.34ba/s]


135693628

In [56]:
dataset_2.map(convert_to_alpaca_format).to_json("datasets/llm2_alpaca.jsonl")

Creating json from Arrow format: 100%|██████████| 26/26 [00:02<00:00, 11.22ba/s]


133884844

In [1]:
from dataset_gen import TopicGenerator
from call_openai_lc import call_openai_api

tg = TopicGenerator(openai_func=call_openai_api)
tg.generate(10, dump=False)

d:\SmartLLM\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\SmartLLM\env\Lib\site-packages\pydantic\_internal\_fields.py:184: UserWarning: Field name "schema" shadows an attribute in parent "BaseModel"; 
  warnings.warn(


TopicGenerator num tokens: 170


['Renewable Energy Sources',
 'Virtual Reality Gaming',
 'Cancer Research Breakthroughs',
 'Cryptocurrency Trends',
 'Mindfulness Meditation Benefits',
 'Global Warming Impact',
 'Artificial Intelligence Ethics',
 'Fashion Trends 2022',
 'Healthy Meal Prep Ideas',
 'Remote Work Productivity Tips']

In [44]:
from dataset_gen import DatasetGenerator, QuestionGenerator

dg = DatasetGenerator()
dg.generate("In the san francisco bay area, does it make sense to rent or buy ?", n=3)

QuestionGenerator num tokens: 240


['What are the current rent versus buy housing market trends in the San Francisco Bay Area for 2023?',
 'Is it more cost-effective to rent or buy a home in the San Francisco Bay Area as of the latest data?',
 'What are the projected long-term financial benefits of renting versus buying in the San Francisco Bay Area?']

In [ ]:
with open('example.txt', 'a+') as file:
    file.write('This is a new line\n')
    file.write('This is another line\n')

    # Move the cursor to the beginning of the file
    file.seek(0)

    # Read the contents of the file
    contents = file.read()
    print(contents)


In [6]:
from models.llm_dataset import LLMDataset, LLMType, Dataset

llm1_data = Dataset.from_json("/home/subhayu/Downloads/SmartLLM/dataset/llm1_alpaca.jsonl")
llm1_data = LLMDataset.from_dataset(llm1_data, LLMType.LLM1)
llm2_data = Dataset.from_json("/home/subhayu/Downloads/SmartLLM/dataset/llm2_alpaca.jsonl")
llm2_data = LLMDataset.from_dataset(llm2_data, LLMType.LLM2)
llm3_data = Dataset.from_json("/home/subhayu/Downloads/SmartLLM/dataset/llm3_alpaca.jsonl")
llm3_data = LLMDataset.from_dataset(llm3_data, LLMType.LLM3)

In [13]:
llm1_datat = llm1_data.get_llm_type_rows(LLMType.LLM1)

In [24]:
llm1_data

LLMDataset(rows=46469)

In [31]:
llm1_datat[0:10].rows

[LLM1DatasetRow(llm=llm1, system=Being an honest and smart assistant talented in breaking down questions into actionable items, you're charged with interpreting a JSON-formatted question. Your output must be a JSON object articulated with two keys: can_i_answer (indicating true if the inquiry is answerable using internal capabilities, or false if it requires external resources) and tasks, delineating the series of steps to answer the question with external aids if can_i_answer is false., input=question='What is the GDP growth rate of India in 2021?', output=question='What is the GDP growth rate of India in 2021?' tasks=['SEARCH,DATA: Retrieve the GDP growth rate data for India for the year 2021'] can_i_answer=False),
 LLM1DatasetRow(llm=llm1, system=In your capacity as a sincere and intelligent assistant expert in breaking down complex queries into accessible steps, your task involves receiving a question in JSON format. You're required to generate a JSON object as a response, featurin